<a href="https://colab.research.google.com/github/kiselpd/HW2_ML/blob/main/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def probability(theta, X):
    result = 1/(1+np.e**(-np.dot(X, theta)))
    return result

#assert type(prob) == np.ndarray, 'Возвращается неверный тип'
#assert prob.shape == (X_data.shape[0],), 'Неверный размер массива'
#assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

In [3]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    result = np.where(prob < threshold, -1, 1)
    return result

#y_pred = binary_class_prediction(theta, X_data)
#assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
#assert y_pred.shape == (X_data.shape[0],), 'Неверный размер массива'
#assert min(y_pred) == -1, 'Функция считается неверно'
#assert max(y_pred) == 1, 'Функция считается неверно'

In [4]:
def logloss(theta, X, y): 
    result=np.sum(np.log(1+np.exp(-(y * (X @ theta))))) / X.shape[0]
    return result

#assert logloss(theta, X_data, y_data).round(3) == 0.861, 'Функция считается неверно'

In [5]:
def gradient(theta, X, y):
    result = np.array(1 / X.shape[0] * np.sum(1 / (1 + np.exp(-(y * (X @ theta)))) * np.exp(-(y * (X @ theta)))) * (y @ X))
    return result

#assert gradient(theta, X_data, y_data).shape == theta.shape, 'Неверный размер массива'

In [6]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')

        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'

        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')

            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [8]:
#X_data, y_data = make_classification(n_samples=2000)
#optimal_theta = fit(X_data, y_data)
#y_pred = binary_class_prediction(optimal_theta, X_data)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
data = pd.read_csv("/content/drive/MyDrive/GoogleColab/HW2/weather.csv")
data.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [56]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [78]:
y = data.RainTomorrow.replace({'No':0, 'Yes': 1})
X = data.drop(columns=["RainTomorrow"]).replace({'No':0, 'Yes': 1})

numeric_data = X.select_dtypes([np.number])
numeric_data_mean = numeric_data.mean()
numeric_features = numeric_data.columns
X = X.fillna(numeric_data_mean)


labelEncoder = LabelEncoder()
X["Date"] = labelEncoder.fit_transform(X["Date"])
X["Location"] = labelEncoder.fit_transform(X["Location"])
X["WindGustDir"] = labelEncoder.fit_transform(X["WindGustDir"])
X["WindDir9am"] = labelEncoder.fit_transform(X["WindDir9am"])
X["WindDir3pm"] = labelEncoder.fit_transform(X["WindDir3pm"])

X_weather, X_test, y_weather, y_test = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=10)

In [80]:
optimal_theta=fit(X_weather.to_numpy(dtype="float64"), y_weather.to_numpy(dtype="float64"), plot=False)

y_pred = binary_class_prediction(optimal_theta, X_test)
print(classification_report(y_pred, y_test))

KeyboardInterrupt: ignored